In [ ]:
# 📦 Installation (compatible Python 3.12)
!pip install torch pyro-ppl numpy pandas matplotlib scikit-learn arviz --quiet


In [ ]:
import pyro
import pyro.distributions as dist
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyro.infer import MCMC, NUTS, SVI, Trace_ELBO
from pyro.optim import Adam
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Simuler données CTR
n_docs = 5
n_sessions = 1000
true_relevance = np.random.beta(2, 5, size=n_docs)
position_bias = np.linspace(1.0, 0.2, n_docs)

click_data = []
for _ in range(n_sessions):
    session = []
    for i in range(n_docs):
        prob = true_relevance[i] * position_bias[i]
        click = np.random.binomial(1, prob)
        session.append(click)
    click_data.append(session)

click_df = pd.DataFrame(click_data, columns=[f'doc_{i+1}' for i in range(n_docs)])
click_totals = click_df.sum(axis=0).values


### Modèle 1 – Régression Logistique Bayésienne (Pyro + NUTS)

In [ ]:
def click_model_1(click_totals, n_docs):
    relevance = pyro.sample("relevance", dist.Beta(torch.ones(n_docs), torch.ones(n_docs)))
    exam_bias = pyro.sample("exam_bias", dist.Beta(torch.ones(n_docs), torch.ones(n_docs)))
    probs = relevance * exam_bias
    with pyro.plate("data", n_docs):
        pyro.sample("clicks", dist.Binomial(total_count=n_sessions, probs=probs), obs=torch.tensor(click_totals))

nuts_kernel = NUTS(click_model_1)
mcmc = MCMC(nuts_kernel, num_samples=500, warmup_steps=200, num_chains=1)
mcmc.run(click_totals, n_docs)

samples = mcmc.get_samples()
relevance_mean = samples["relevance"].mean(0).numpy()
exam_mean = samples["exam_bias"].mean(0).numpy()

plt.plot(true_relevance, 'o-', label="Pertinence réelle")
plt.plot(relevance_mean, 's--', label="Estimation NUTS")
plt.title("Pertinence - Modèle 1")
plt.legend()
plt.show()


### Modèle 2 – Réseau Bayésien avec SVI (Forme scalaire)

In [ ]:
def model_nn_scalar(click_count, position):
    weight = pyro.sample("weight", dist.Normal(0., 1.))
    bias = pyro.sample("bias", dist.Normal(0., 1.))
    prob = torch.sigmoid(weight * position + bias)
    pyro.sample("click", dist.Binomial(total_count=n_sessions, probs=prob),
                obs=torch.tensor(click_count, dtype=torch.float32))

def guide_nn_scalar(click_count, position):
    weight_loc = pyro.param("weight_loc", torch.tensor(0.))
    weight_scale = pyro.param("weight_scale", torch.tensor(1.), constraint=dist.constraints.positive)
    bias_loc = pyro.param("bias_loc", torch.tensor(0.))
    bias_scale = pyro.param("bias_scale", torch.tensor(1.), constraint=dist.constraints.positive)
    pyro.sample("weight", dist.Normal(weight_loc, weight_scale))
    pyro.sample("bias", dist.Normal(bias_loc, bias_scale))

results = []
for i in range(n_docs):
    pyro.clear_param_store()
    click_count = click_totals[i]
    position = position_bias[i]
    svi = SVI(model_nn_scalar, guide_nn_scalar, Adam({"lr": 0.01}), loss=Trace_ELBO())
    for step in range(1000):
        svi.step(click_count, position)
    w = pyro.param("weight_loc").item()
    b = pyro.param("bias_loc").item()
    results.append((w, b))

weights = [w for w, _ in results]
plt.bar(range(1, n_docs + 1), weights)
plt.title("Poids estimés (SVI)")
plt.xlabel("Document")
plt.ylabel("Poids")
plt.show()


### Modèle 3 – Classifieur Naïf Bayésien

In [ ]:
X = np.column_stack([position_bias] * n_sessions)
y = np.array(click_data).flatten()
X = X.flatten().reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model_nb = GaussianNB()
model_nb.fit(X_train, y_train)
y_pred = model_nb.predict(X_test)
acc_nb = accuracy_score(y_test, y_pred)
print(f"🎯 Précision du modèle Naïf Bayésien : {acc_nb:.2%}")


### Modèle 4 – Arbre de Décision

In [ ]:
model_tree = DecisionTreeClassifier(max_depth=3, random_state=0)
model_tree.fit(X_train, y_train)
y_pred_tree = model_tree.predict(X_test)
acc_tree = accuracy_score(y_test, y_pred_tree)
print(f"🎯 Précision de l'arbre de décision : {acc_tree:.2%}")
